In [31]:
import os
import pandas as pd 
import numpy as np 
import tensorflow as tf 

In [ ]:
head_columns = ['user', 'activity', 'time', 'x', 'y', 'z']

In [43]:
for folder_divice in os.listdir(r'D:\bangkit\capstone\Family_health_Tracker\ML_workspace\daily_activity\raw'):
    for folder_move in os.listdir(os.path.join(folder_divice,r'D:\bangkit\capstone\Family_health_Tracker\ML_workspace\daily_activity\raw')):
        for dirname,_,filename in os.walk(folder_move):
            print(filename)

In [ ]:
def data_integ(columns,input_folder,output_folder):
    sum_data = pd.DataFrame(data=None,columns=columns)
    for folder_divice in os.listdir(input_folder):
        for folder_move in os.listdir(os.path.join(input_folder,folder_divice)):
            path_fmove = os.path.join(input_folder,folder_divice,folder_move)
            for file in os.listdir(path_fmove):
                df = pd.read_csv(os.path.join(path_fmove,file),sep=',',header=None)
                temp_data = pd.DataFrame(data=df.values, columns=columns)
                sum_data = pd.concat([sum_data,temp_data])